# [모의 캐글 - 게임] 비매너 댓글 식별 

- 자연어 multi label classification 과제

참고 논문 : 
- [BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

# 1. 환경 설정 및 라이브러리 불러오기

In [1]:
# !pip install -r requirements.txt

In [1]:
import pandas as pd
import os
import json
import numpy as np
import shutil

from sklearn.metrics import f1_score
from datetime import datetime, timezone, timedelta
import random
from tqdm import tqdm

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW

from transformers import logging, get_linear_schedule_with_warmup


from transformers import ( 
    BertConfig,
    ElectraConfig
)

from transformers import (
    BertTokenizer,  
    AutoTokenizer,
    ElectraTokenizer,
    AlbertTokenizer
)

from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification,
    ElectraModel
)


In [5]:
# 사용할 GPU 지정
print("number of GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
print("Does GPU exist? : ", use_cuda)
DEVICE = torch.device("cuda" if use_cuda else "cpu")

number of GPUs:  1
Does GPU exist? :  True


In [6]:
# True 일 때 코드를 실행하면 example 등을 보여줌
DEBUG = True

In [7]:
# config 파일 불러오기
config_path = os.path.join('config.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = json.load(f)
            print("config file loaded.")
            print(args['pretrained_model'])
    else:
        assert False, 'config json file cannot be found.. please check the path again.'
    
    return args


# 코드 중간중간에 끼워넣어 리셋 가능
args = set_config(config_path)

# 결과 저장 폴더 미리 생성
os.makedirs(args['result_dir'], exist_ok=True)
os.makedirs(args['config_dir'], exist_ok=True)

config file loaded.
beomi/KcELECTRA-base


config_bias1_adamW.json
```json
{
    "data_dir": "./data",
    "result_dir": "./result/",
    "config_dir": "./exp_config/",
    "pretrained_model": "beomi/KcELECTRA-base",  
    "architecture": "ElectraForSequenceClassification",
    "tokenizer_class": "BertTokenizer",
    "num_classes": 3,
    "max_seq_len": 128,
    "train_epochs": 8,
    "adam_epsilon": 1e-8,
    "seed": 42,
    "train_batch_size": 64,
    "eval_batch_size": 128,
    "learning_rate": 5e-5,
    "warmup_proportion": 0,
    "run": "bias1_adamW",
    "patience": 10
}
```

config_hate2_adamW.json
```json
{
    "data_dir": "./data",
    "result_dir": "./result/",
    "config_dir": "./exp_config/",
    "pretrained_model": "beomi/KcELECTRA-base",  
    "architecture": "ElectraForSequenceClassification",
    "tokenizer_class": "BertTokenizer",
    "num_classes": 2,
    "max_seq_len": 128,
    "train_epochs": 8,
    "adam_epsilon": 1e-8,
    "seed": 42,
    "train_batch_size": 64,
    "eval_batch_size": 128,
    "learning_rate": 5e-5,
    "warmup_proportion": 0,
    "run": "hate2_adamW",
    "patience": 10
}
```

# 2. EDA 및 데이터 전처리

In [8]:
# data 경로 설정  
train_path = os.path.join(args['data_dir'],'train.csv')

print("train 데이터 경로가 올바른가요? : ", os.path.lexists(train_path))


train 데이터 경로가 올바른가요? :  True


### 2-1. Train 데이터 확인

In [9]:
train_df = pd.read_csv(train_path, encoding = 'UTF-8-SIG')

train_df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


In [8]:
### v2 에서 추가됨

# title 중 가장 긴 타이틀 길이
max_len_title = np.max(train_df['title'].str.len())
max_len_title

63

In [9]:
# comment 중 가장 긴 타이틀 길이
max_len_comment=np.max(train_df['comment'].str.len())
max_len_comment

137

In [10]:
# 길이가 128이 넘는 코멘트 확인
train_df['comment'][train_df['comment'].str.len()>128]

149     강혁민한테 뭐라 할 필요는 없는데 ㅋㅋ 왜 과거에 친하지 않은 동료가 있었는ㄷㅔ 그...
1437    그냥 좀 착하게 살면 안되겠냐? 여기다가 설정이네 어쩌네 썰전 펼치면서 애 하나 잡...
2024    전진 씨, 붐 씨에게 사과하세요. 댁이 붐 사칭하며 나이트클럽에서 놀고 싸인해주며 ...
3031    팩트 = 전처 언니라는 사람이 인터넷에 폭로글 쓴 후, 몇 시간 뒤 자진삭제. 여전...
3068    솔까 마닷 욕 하는 인간들 치고 돈 안 떼인 인간 없지 싶다 안그러고는 불구대천 원...
5834    쓰레기에 대해 그만 쓰고 장자연 사건에 대해 써주세요. 더 의미 있고 도 큰 사건일...
5979    안재현 망 할 놈 얼마나 개망나니 같이 행동했으면 구혜선이 한를 품를까. 구혜선 인...
6070    니놈이 실력이 있냐 외모가 출중하냐 호감형이길 하냐 키라도 크냐 천운으로 이름 좀 ...
7276    음주운전 또 면죄부? 미친 방송국 놈들. 호란이야 아쉬웠겠지만 시청자는 아쉽지 않은...
Name: comment, dtype: object

In [10]:
len(train_df)

8367

In [12]:
print("bias classes: ", train_df.bias.unique())
print("hate classes: ", train_df.hate.unique())

bias classes:  ['none' 'others' 'gender']
hate classes:  ['none' 'hate']


In [13]:
pd.crosstab(train_df.bias, train_df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


In [14]:
pd.crosstab(train_df.bias, train_df.hate, margins=True, normalize=True)

hate,hate,none,All
bias,,,
gender,0.145333,0.009920,0.155253
none,0.247161,0.408988,0.656149
others,0.171746,0.016852,0.188598
All,0.564240,0.435760,1.000000


### 2-2. Test 데이터 확인

In [11]:
test_path = os.path.join(args['data_dir'],'test.csv')
print("test 데이터 경로가 올바른가요? : ", os.path.lexists(test_path))

test 데이터 경로가 올바른가요? :  True


In [24]:
test_df = pd.read_csv(test_path)
test_df.head()

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...


In [17]:
len(test_df)

511

### 2-3. 데이터 전처리 (Label Encoding)
bias, hate 라벨들의 class를 정수로 변경하여 라벨 인코딩을 하기 위한 딕셔너리입니다.

In [12]:
LABEL2ID_BIAS = {'none': 0, 'gender': 1, 'others': 2}
LABEL2ID_HATE = {'none': 0, 'hate': 1}

In [13]:
train_df['bias_label'] = train_df['bias'].apply(lambda x: LABEL2ID_BIAS[x])
train_df['hate_label'] = train_df['hate'].apply(lambda x: LABEL2ID_HATE[x])
train_df[['bias_label', 'hate_label']]

,bias_label,hate_label
0,0,0
1,0,1
2,2,1
3,0,0
4,0,0
...,...,...
8362,2,1
8363,1,1
8364,0,0
8365,0,0


# 3. Dataset 로드

## 3-0. Pre-trained tokenizer 탐색

In [14]:
# config.json 에서 지정 이름별로 가져올 라이브러리 지정

TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer,
    "AutoTokenizer": AutoTokenizer,
    "ElectraTokenizer": ElectraTokenizer,
    "AlbertTokenizer": AlbertTokenizer
}


- Tokenizer 사용 예시

In [15]:
tokenizer = TOKENIZER_CLASSES[args['tokenizer_class']].from_pretrained(args['pretrained_model'])
if DEBUG==True:
    print(tokenizer)

PreTrainedTokenizer(name_or_path='beomi/KcELECTRA-base', vocab_size=50135, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [16]:
if DEBUG == True:
    example = train_df['title'][0]
    comment_ex = train_df['comment'][0]
    print(tokenizer(example, comment_ex))

{'input_ids': [2, 6, 11, 24787, 2089, 5146, 4028, 11, 1709, 4071, 4069, 16, 20778, 4177, 4331, 8069, 35054, 12634, 47185, 13182, 5, 10491, 33, 6, 3, 20778, 4177, 8057, 11061, 4479, 4025, 7997, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
if DEBUG==True:
    print(tokenizer.encode(example), end="\n")
    
    # 토큰으로 나누기
    print(tokenizer.tokenize(example), end="\n")
    
    # 토큰 id로 매핑하기
    print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(example)))


[2, 6, 11, 24787, 2089, 5146, 4028, 11, 1709, 4071, 4069, 16, 20778, 4177, 4331, 8069, 35054, 12634, 47185, 13182, 5, 10491, 33, 6, 3]
['"', "'", '미스터', '션', '##샤', '##인', "'", '변', '##요', '##한', ',', '김태', '##리', '##와', '같은', '양복', '입고', '학당', '방문', '!', '이유는', '?', '"']
[6, 11, 24787, 2089, 5146, 4028, 11, 1709, 4071, 4069, 16, 20778, 4177, 4331, 8069, 35054, 12634, 47185, 13182, 5, 10491, 33, 6]


## 3-1. Dataset 만드는 함수 정의

In [18]:
class CustomDataset(Dataset):

    def __init__(self, df, tokenizer, max_len, mode = 'train'):

        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        
        if self.mode != 'test':
            try: 
                self.bias_labels = df['bias_label'].tolist()
                self.hate_labels = df['hate_label'].tolist()
            except:
                assert False, 'CustomDataset Error : label columns does not exist in the dataframe'
     
    def __len__(self):
        return len(self.data)
                

    def __getitem__(self, idx):
        """
        전체 데이터에서 특정 인덱스 (idx)에 해당하는 기사제목과 댓글 내용을 
        토크나이즈한 data('input_ids', 'attention_mask','token_type_ids')의 딕셔너리 형태로 불러옴
        """
        # title = self.data.title.iloc[idx]
        comment = self.data.comment.iloc[idx]
        
        tokenized_text = self.tokenizer(
            # title, 
            comment,
            padding= 'max_length',
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors = "pt"
        )
        
        data = {'input_ids': tokenized_text['input_ids'].clone().detach().long().squeeze(0),
               'attention_mask': tokenized_text['attention_mask'].clone().detach().long().squeeze(0),
               'token_type_ids': tokenized_text['token_type_ids'].clone().detach().long().squeeze(0),
               }
        
        if self.mode != 'test':
            bias_label = self.bias_labels[idx]
            hate_label = self.hate_labels[idx]
            return data, bias_label, hate_label
        else:
            return data
        

    
train_dataset = CustomDataset(train_df, tokenizer, args['max_seq_len'], mode ='train')
print("train dataset loaded.")

train dataset loaded.


In [19]:
if DEBUG ==True :
    print("dataset sample : ")
    print(train_dataset[0])

dataset sample : 
({'input_ids': tensor([    2, 20778,  4177,  8057, 11061,  4479,  4025,  7997,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

In [26]:
# encoded_plus = tokenizer.encode_plus(
#                     sentence,                      # Sentence to encode.
#                     add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                     max_length = 128,           # Pad & truncate all sentences.
#                     pad_to_max_length = True,
#                     return_attention_mask = True,   # Construct attention masks.
#                     return_tensors = 'pt',     # Return pytorch tensors.
#                )

## 3-2. Train, Validation set 나누기

In [20]:
from sklearn.model_selection import train_test_split
                                                         
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args['seed'], stratify=train_df.apply(lambda x: (x['bias'], x['hate']), axis=1))

train_dataset = CustomDataset(train_data, tokenizer, args['max_seq_len'], 'train')
val_dataset = CustomDataset(val_data, tokenizer, args['max_seq_len'], 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

Train dataset:  7530
Validation dataset:  837


In [28]:
pd.crosstab(train_data.bias, train_data.hate, margins=True, normalize=True)

hate,hate,none,All
bias,,,
gender,0.145286,0.009960,0.155246
none,0.247145,0.409031,0.656175
others,0.171713,0.016866,0.188579
All,0.564143,0.435857,1.000000


In [29]:
pd.crosstab(val_data.bias, val_data.hate, margins=True, normalize=True)

hate,hate,none,All
bias,,,
gender,0.145759,0.009558,0.155317
none,0.247312,0.408602,0.655914
others,0.172043,0.016726,0.188769
All,0.565114,0.434886,1.000000


In [30]:
train_dataset_full = CustomDataset(train_df, tokenizer, args['max_seq_len'], 'train')

# 4. 분류 모델 학습을 위한 세팅

## 4-1. 아키텍쳐 설정




- [PretrainedConfig](https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/configuration#transformers.PretrainedConfig.from_pretrained)
-[KcELECTRA 사전학습 모델](https://github.com/Beomi/KcELECTRA)

In [23]:
from transformers import logging
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification,
    "AutoModel": AutoModel,
    "ElectraForSequenceClassification": ElectraForSequenceClassification,
    "AlbertForSequenceClassification": AlbertForSequenceClassification,
    "ElectraModel": ElectraModel
}


model = BASE_MODELS[args['architecture']].from_pretrained(args['pretrained_model'], 
                                                         num_labels = args['num_classes'], 
                                                        #  output_attentions = False, # Whether the model returns attentions weights.
                                                        #  output_hidden_states = True # Whether the model returns all hidden-states.
                                                        )
if DEBUG==True:
    # 모델 구조 확인
    print(model)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(50135, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

### 4-2. 모델 구성 확인

In [26]:
if DEBUG==True:
    params = list(model.named_parameters())

    print('The ELECTRA model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The ELECTRA model has 201 different named parameters.

==== Embedding Layer ====

electra.embeddings.word_embeddings.weight               (50135, 768)
electra.embeddings.position_embeddings.weight             (512, 768)
electra.embeddings.token_type_embeddings.weight             (2, 768)
electra.embeddings.LayerNorm.weight                           (768,)
electra.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

electra.encoder.layer.0.attention.self.query.weight       (768, 768)
electra.encoder.layer.0.attention.self.query.bias             (768,)
electra.encoder.layer.0.attention.self.key.weight         (768, 768)
electra.encoder.layer.0.attention.self.key.bias               (768,)
electra.encoder.layer.0.attention.self.value.weight       (768, 768)
electra.encoder.layer.0.attention.self.value.bias             (768,)
electra.encoder.layer.0.attention.output.dense.weight     (768, 768)
electra.encoder.layer.0.attention.output.dense.bias         

# 5. 학습 진행

## 5-0. Early Stopper 함수 정의

- v2에서 코드 일부 삭제

In [61]:
class LossEarlyStopper():
    """Early stopper

        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        # 첫 에폭
        if self.min_loss == np.Inf:
            self.min_loss = loss
           
        # loss가 줄지 않는다면 -> patience_counter 1 증가
        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            # patience 만큼 loss가 줄지 않았다면 학습을 중단합니다.
            if self.patience_counter == self.patience:
                self.stop = True
            print(msg)
        # loss가 줄어듬 -> min_loss 갱신, patience_counter 초기화
        elif loss <= self.min_loss:
            self.patience_counter = 0
            ### v2 에서 수정됨
            ### self.save_model = True -> 삭제 (사용하지 않음)
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss

            print(msg)

## 5-1. Epoch 별 학습 및 검증

- [Transformers optimization documentation](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules)
- [스케줄러 documentation](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#schedules)
- Adam optimizer의 epsilon 파라미터 eps = 1e-8 는 "계산 중 0으로 나눔을 방지 하기 위한 아주 작은 숫자 " 입니다. ([출처](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/))
- 스케줄러 파라미터
    - `warmup_ratio` : 
      - 학습이 진행되면서 학습률을 그 상황에 맞게 가변적으로 적당하게 변경되게 하기 위해 Scheduler를 사용합니다.
      - 처음 학습률(Learning rate)를 warm up하기 위한 비율을 설정하는 warmup_ratio을 설정합니다.
  

In [ ]:
args = set_config(config_path)

logging.set_verbosity_warning()

# 재현을 위해 모든 곳의 시드 고정
seed_val = args['seed']
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def train(model, train_data, val_data, args, mode = 'train'):
    
    # args.run은 실험 이름 (어디까지나 팀원들간의 버전 관리 및 공유 편의를 위한 것으로, 자유롭게 수정 가능합니다.)
    print("RUN : ", args['run'])
    shutil.copyfile("config.json", os.path.join(args['config_dir'], f"config_{args['run']}.json"))

    early_stopper = LossEarlyStopper(patience=args['patience'])
    
    train_dataloader = DataLoader(train_data, batch_size=args['train_batch_size'], shuffle=True)
    val_dataloader = DataLoader(val_data, batch_size=args['train_batch_size'])

    
    if DEBUG == True:
        # 데이터로더가 성공적으로 로드 되었는지 확인
        for idx, data in enumerate(train_dataloader):
            if idx==0:
                print("batch size : ", len(data[0]['input_ids']))
                print("The first batch looks like ..\n", data[0])
    
    
    # criterion = nn.BCEWithLogitsLoss()
    
    total_steps = len(train_dataloader) * args['train_epochs']

    ### v2에서 수정됨 (Adam -> AdamW)
    optimizer = AdamW(model.parameters(), lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * args['warmup_proportion']), 
                                                num_training_steps=total_steps)

    
    if use_cuda:
        model = model.to(DEVICE)
        # criterion = criterion.to(DEVICE)
        

    tr_loss = 0.0
    val_loss = 0.0
    best_score = 0.0
    best_loss = np.inf
      

    for epoch_num in range(args['train_epochs']):

            total_acc_train = 0
            total_loss_train = 0
            
            assert mode in ['train', 'val'], 'your mode should be either \'train\' or \'val\''
            
            if mode =='train':
                for train_input, train_label, _ in tqdm(train_dataloader):
                    train_input = {k: v.to(DEVICE) for k, v in train_input.items()}
                    train_label = train_label.long().to(DEVICE)  
                    
                    output = model(**train_input, labels=train_label)
                    batch_loss = output.loss
                    total_loss_train += batch_loss.item()

                    acc = (output.logits.argmax(dim=1) == train_label).sum().item()
                    total_acc_train += acc
                    
                    ### v2에 수정됨
                    optimizer.zero_grad()
                    
                    batch_loss.backward()
                    optimizer.step()
                    
                    ### v2 에 수정됨
                    scheduler.step()
                    

            total_acc_val = 0
            total_loss_val = 0
            
            # validation을 위해 이걸 넣으면 이 evaluation 프로세스 중엔 dropout 레이어가 다르게 동작한다.
            model.eval()
            
            with torch.no_grad():

                for val_input, val_label, _ in val_dataloader:
                    val_input = {k: v.to(DEVICE) for k, v in val_input.items()}
                    val_label = val_label.long().to(DEVICE)

                    output = model(**val_input, labels=val_label)
                    batch_loss = output.loss
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.logits.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            
            train_loss = total_loss_train / len(train_data)
            train_accuracy = total_acc_train / len(train_data)
            val_loss = total_loss_val / len(val_data)
            val_accuracy = total_acc_val / len(val_data)
            
            # 한 Epoch 학습 후 학습/검증에 대해 loss와 평가지표 (여기서는 accuracy로 임의로 설정) 출력
            print(
                f'Epoch: {epoch_num + 1} \
                | Train Loss: {train_loss: .3f} \
                | Train Accuracy: {train_accuracy: .3f} \
                | Val Loss: {val_loss: .3f} \
                | Val Accuracy: {val_accuracy: .3f}')
          
            # early_stopping check
            early_stopper.check_early_stopping(loss=val_loss)

            if early_stopper.stop:
                print('Early stopped, Best score : ', best_score)
                break

            ### v2 에 수정됨
            ### loss와 accuracy가 꼭 correlate하진 않습니다.
            ### 
            ### 원본 (필요하다면 다시 해제 후 사용)
            # if val_accuracy > best_score : 
            if val_loss < best_loss :
            # 모델이 개선됨 -> 검증 점수와 베스트 loss, weight 갱신
                best_score = val_accuracy 
                
                ### v2에서 추가
                best_loss = val_loss
                # 학습된 모델을 저장할 디렉토리 및 모델 이름 지정
                SAVED_MODEL =  os.path.join(args['result_dir'], f'best_{args["run"]}.pt')
            
                check_point = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                }
                torch.save(check_point, SAVED_MODEL)  
              
            # print("scheduler : ", scheduler.state_dict())


    print("train finished")


train(model, train_dataset_full, val_dataset, args, mode = 'train')

# 6. Test dataset으로 추론 (Prediction)

In [29]:
# 테스트 데이터셋 불러오기
test_dataset = CustomDataset(test_df, tokenizer=tokenizer, max_len=args['max_seq_len'], mode='test')

def test(model, saved_model, test_dataset, args):


    test_dataloader = DataLoader(test_dataset, batch_size=args['eval_batch_size'])


    if use_cuda:

        model = model.to(DEVICE)
        model.load_state_dict(torch.load(saved_model)['model'])


    model.eval()

    pred = []
    pred_probs = []

    with torch.no_grad():
        for test_input in test_dataloader:
            test_input = {k: v.to(DEVICE) for k, v in test_input.items()}
            output = model(**test_input)

            pred += output.logits.argmax(dim=1).cpu().tolist()
            pred_probs += output.logits.sigmoid().cpu().tolist()
                
    return pred, pred_probs

SAVED_MODEL =  os.path.join(args['result_dir'], f'best_{args["run"]}.pt')

pred, pred_probs = test(model, SAVED_MODEL, test_dataset, args)

In [30]:
print("prediction completed for ", len(pred), "comments")


prediction completed for  511 comments


In [36]:
# 라벨 값 별로 bias, hate로 디코딩 하기 위한 list
LABELS_BIAS = ['none', 'gender', 'others']
LABELS_HATE = ['none', 'hate']

# from transformers import pipeline

# pipe_bias = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-bias', device=0)

# 인코딩 값으로 나온 타겟 변수를 디코딩
pred_bias = [LABELS_BIAS[x] for x in pred]

# config_hate2_adamW.json로 학습 + threshold moving 사용한 hate label 사용
submit_hate = pd.read_csv(os.path.join(args['result_dir'], 'submission_hate2_adamW_v2.csv'))
pred_hate = submit_hate['hate']
print('decode Completed!')



decode Completed!


In [2]:
submit = pd.read_csv(os.path.join('sample_submission.csv'))
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,none
3,3,none,none
4,4,none,none
...,...,...,...
506,506,none,none
507,507,none,none
508,508,none,none
509,509,none,none


In [72]:
submit['bias'] = pred_bias
submit['hate'] = pred_hate
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,others,hate


In [73]:
submit.to_csv(os.path.join(args['result_dir'], f"submission_{args['run']}_v2.csv"), index=False)

## Threshold Moving

In [27]:
SAVED_MODEL =  os.path.join(args['result_dir'], 'best_hate2_adamW.pt')

In [44]:
from sklearn.metrics import f1_score

_val_dataset = CustomDataset(val_data, tokenizer=tokenizer, max_len=args['max_seq_len'], mode='test')
pred_val, pred_probs_val = test(model, SAVED_MODEL, _val_dataset, args)

thresholds = np.linspace(0, 1, 1000, endpoint=False)
index_threshold = np.argmax([f1_score(val_data['hate'], ['hate' if x[1] >= t else 'none' for x in pred_probs_val], average='macro') for t in thresholds])
index_threshold

98

In [45]:
threshold = thresholds[index_threshold]
threshold

0.098

In [46]:
f1_score(val_data['hate'], [LABELS_HATE[x] for x in pred_val], average='macro')

1.0

In [47]:
f1_score(val_data['hate'], ['hate' if x[1] >= threshold else 'none' for x in pred_probs_val], average='macro')

1.0

In [60]:
submit = pd.read_csv(os.path.join('sample_submission.csv'))

submit['bias'] = pred_bias
submit['hate'] = ['hate' if x[1] >= threshold else 'none' for x in pred_probs]

submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,gender,hate
3,3,gender,hate
4,4,gender,hate
...,...,...,...
506,506,gender,hate
507,507,none,none
508,508,gender,hate
509,509,none,hate


In [ ]:
submit.to_csv(os.path.join(args['result_dir'], f"submission_{args['run']}_v3.csv"), index=False)

# 7. Hugging Face Pretrained Model 그대로 사용해보기

- https://github.com/kocohub/korean-hate-speech

In [21]:
submit = pd.read_csv('sample_submission.csv')
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,none
3,3,none,none
4,4,none,none
...,...,...,...
506,506,none,none
507,507,none,none
508,508,none,none
509,509,none,none


In [21]:
from transformers import pipeline

pipe_bias = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-bias', device=0)
pipe_hate = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-hate', device=0)

In [61]:
submit['bias'] = [x['label'] for x in pipe_bias(test_df['comment'].to_list())]
submit['hate'] = ['none' if x['label'] == 'none' else 'hate' for x in pipe_hate(test_df['comment'].to_list())]

In [23]:
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,none
507,507,none,none
508,508,others,hate
509,509,others,hate


In [23]:
submit.to_csv(os.path.join(args['result_dir'], f"submission_v1.csv"), index=False)

## Threshold Moving

In [85]:
pipe_hate = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-hate', device=0, return_all_scores=True)

outputs_hate = pipe_hate(test_df['comment'].to_list())
outputs_hate

[[{'label': 'none', 'score': 0.9925063252449036},
  {'label': 'offensive', 'score': 0.005888822954148054},
  {'label': 'hate', 'score': 0.0016048253746703267}],
 [{'label': 'none', 'score': 0.9734423160552979},
  {'label': 'offensive', 'score': 0.024174880236387253},
  {'label': 'hate', 'score': 0.0023828204721212387}],
 [{'label': 'none', 'score': 0.03134765848517418},
  {'label': 'offensive', 'score': 0.8452329039573669},
  {'label': 'hate', 'score': 0.12341945618391037}],
 [{'label': 'none', 'score': 0.0021730673033744097},
  {'label': 'offensive', 'score': 0.021381236612796783},
  {'label': 'hate', 'score': 0.976445734500885}],
 [{'label': 'none', 'score': 0.44181060791015625},
  {'label': 'offensive', 'score': 0.5264903903007507},
  {'label': 'hate', 'score': 0.031698983162641525}],
 [{'label': 'none', 'score': 0.11184421926736832},
  {'label': 'offensive', 'score': 0.8434919714927673},
  {'label': 'hate', 'score': 0.04466378688812256}],
 [{'label': 'none', 'score': 0.023514250293

In [86]:
probs_hate = [[x[0]['score'], x[1]['score'] + x[2]['score']] for x in outputs_hate]
probs_hate

[[0.9925063252449036, 0.007493648328818381],
 [0.9734423160552979, 0.02655770070850849],
 [0.03134765848517418, 0.9686523601412773],
 [0.0021730673033744097, 0.9978269711136818],
 [0.44181060791015625, 0.5581893734633923],
 [0.11184421926736832, 0.8881557583808899],
 [0.02351425029337406, 0.9764856845140457],
 [0.01978875696659088, 0.9802112877368927],
 [0.04309116303920746, 0.9569088071584702],
 [0.9926250576972961, 0.0073749563889577985],
 [0.8000229597091675, 0.1999770738184452],
 [0.0025073818396776915, 0.9974925797432661],
 [0.9926959276199341, 0.0073040800634771585],
 [0.0020401000510901213, 0.9979598745703697],
 [0.9926194548606873, 0.007380579016171396],
 [0.0024721319787204266, 0.9975279159843922],
 [0.1638796627521515, 0.8361202627420425],
 [0.03738470375537872, 0.9626153260469437],
 [0.8511348366737366, 0.14886507391929626],
 [0.992743968963623, 0.007256064331158996],
 [0.002428857609629631, 0.9975711386650801],
 [0.05998794734477997, 0.940012015402317],
 [0.0205257199704647

In [87]:
df_dev = pd.read_csv('data/dev.tsv', sep='\t')
df_dev

,comments,contain_gender_bias,bias,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,False,none,none
1,지현우 나쁜놈,False,none,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,False,none,hate
3,설마 ㅈ 현정 작가 아니지??,True,gender,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,False,none,offensive
...,...,...,...,...
466,지현우 범죄 저지르지 않았나요?,False,none,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,True,gender,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,False,others,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",True,gender,hate


In [88]:
df_dev['hate'] = df_dev['hate'].apply(lambda x: 'none' if x == 'none' else 'hate')
df_dev

,comments,contain_gender_bias,bias,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,False,none,none
1,지현우 나쁜놈,False,none,hate
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,False,none,hate
3,설마 ㅈ 현정 작가 아니지??,True,gender,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,False,none,hate
...,...,...,...,...
466,지현우 범죄 저지르지 않았나요?,False,none,hate
467,여자인생 망칠 일 있나 ㅋㅋ,True,gender,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,False,others,hate
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",True,gender,hate


In [90]:
outputs_hate_dev = pipe_hate(df_dev['comments'].to_list())
probs_hate_dev = [[x[0]['score'], x[1]['score'] + x[2]['score']] for x in outputs_hate_dev]
probs_hate_dev

[[0.9926590323448181, 0.007340976851992309],
 [0.15577200055122375, 0.8442279435694218],
 [0.0273718424141407, 0.9726282209157944],
 [0.00586236035451293, 0.9941376447677612],
 [0.04578247293829918, 0.9542175605893135],
 [0.0021454819943755865, 0.9978544842451811],
 [0.9926667809486389, 0.007333182264119387],
 [0.9925587177276611, 0.007441234076395631],
 [0.37537360191345215, 0.6246264073997736],
 [0.2643817663192749, 0.7356183081865311],
 [0.979048490524292, 0.020951526006683707],
 [0.006837398745119572, 0.993162602186203],
 [0.9923880100250244, 0.007612010813318193],
 [0.03323400020599365, 0.9667659997940063],
 [0.02889474853873253, 0.9711052626371384],
 [0.02448234148323536, 0.975517749786377],
 [0.003995600622147322, 0.9960043281316757],
 [0.0019946908578276634, 0.9980052597820759],
 [0.019437290728092194, 0.980562686920166],
 [0.9229986071586609, 0.0770013933070004],
 [0.8442890644073486, 0.15571095701307058],
 [0.037372246384620667, 0.9626277983188629],
 [0.001979632070288062, 0.

In [110]:
from sklearn.metrics import f1_score

thresholds = np.linspace(0, 0.5, 1000, endpoint=False)
index_threshold = np.argmax([f1_score(df_dev['hate'], ['hate' if x[1] >= t else 'none' for x in probs_hate_dev], average='macro') for t in thresholds])
threshold = thresholds[index_threshold]
threshold

0.117

In [111]:
f1_score(df_dev['hate'], ['hate' if x[1] >= threshold else 'none' for x in probs_hate_dev], average='macro')

0.8342922185430464

In [112]:
submit = pd.read_csv('sample_submission.csv')
submit['bias'] = test_df['comment'].map(lambda x: pipe_bias(x)[0]['label'])
submit['hate'] = ['hate' if x[1] >= threshold else 'none' for x in probs_hate]
submit.to_csv(os.path.join(args['result_dir'], f"submission_v2.csv"), index=False)

/home/jmkim/mambaforge/lib/python3.9/site-packages/transformers/pipelines/base.py:1075: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [72]:
submit = pd.read_csv('sample_submission.csv')
submit['bias'] = test_df['comment'].map(lambda x: pipe_bias(x)[0]['label'])
submit['hate'] = ['hate'] * 511
submit.to_csv(os.path.join(args['result_dir'], f"submission_all_hate.csv"), index=False)

/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:976: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [73]:
submit = pd.read_csv('sample_submission.csv')
submit['bias'] = test_df['comment'].map(lambda x: pipe_bias(x)[0]['label'])
submit['hate'] = ['none'] * 511
submit.to_csv(os.path.join(args['result_dir'], f"submission_all_none.csv"), index=False)

/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:976: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
